# DTSC670: Foundations of Machine Learning Models

## Assignment: Johnny Likes Pie

#### Name: Arsenika Merenkov

### Details

First, make sure that you watch the video titled "Should You Play Golf Today" in the "Preparation for Assignment 1" section of Brightspace.  This assignment is meant to purely allow you to perform some basic steps with Scikit-Learn to get you used to working with it.

The following data describes features of different types of pie, along with a positive or negative classification of the pie based whether or not Johnny likes it.  A positive classification means Johnny likes that pie; a negative classification means Johnny does not like that pie.

<img src="JohnnyPies.png " width ="600" />

### Import Data

Let's start out by importing some standard imports.

In [1]:
# Importing numpy and pandas library
import numpy as np
import pandas as pd

# Set the maximum number of columns to None to display all columns in the DataFrame
# Do not change these options; This allows the CodeGrade auto grading to function correctly
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore') 

Next you should place the data file called `JohnnyPiesData.csv` and this Jupyter notebook in the same directory.  Use the [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function to read in the data from the comma-separated values (csv) file to a Pandas DataFrame called `pie_df` and output the data to take a look.

In [2]:
# Store the file name of the data in a variable
fileName = "JohnnyPiesData.csv"

# Read the csv file and store it in a dataframe
pie_df = pd.read_csv(fileName)
pie_df

,Example,Crust Shape,Crust Size,Crust Shade,Filling Size,Filling Shade,Class
0,ex1,Circle,Thick,Gray,Thick,Dark,pos
1,ex2,Circle,Thick,White,Thick,Dark,pos
2,ex3,Triangle,Thick,Dark,Thick,Gray,pos
3,ex4,Circle,Thin,White,Thin,Dark,pos
4,ex5,Square,Thick,Dark,Thin,White,pos
5,ex6,Circle,Thick,White,Thin,Dark,pos
6,ex7,Circle,Thick,Gray,Thick,White,neg
7,ex8,Square,Thick,White,Thick,Gray,neg
8,ex9,Triangle,Thin,Gray,Thin,Dark,neg
9,ex10,Circle,Thick,Dark,Thick,White,neg


## Prepare Data for Linear Regression

- Drop the `Example` column from the `pie_df` DataFrame, because it offers no information.

- Encode all categorical data into numeric data via the "One Hot Encoding" technique provided by the Pandas [get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) function.  

  - Since we are performing ordinary least squares linear regression, we will want to drop one of the newly created Boolean-valued features (output from the `get_dummies()` function) to prevent introducing unwanted correlation in the data.  Include `drop_first = True` as an argument to the `get_dummies()` function.

- Store the final features in a DataFrame called `features`.  The one-hot-encoded columns must go in the same order as the original data so that the linear regression coefficients match what CodeGrade is expecting.

- Store the positive class labels in a DataFrame called `response`.  The `response` data must be a DataFrame and not a Series or some of the code towards the end of this notebook may not function correctly and your output might be slightly different than what CodeGrade is expecting.

**Note:** Since we are not concerned with generalization error in this assignment, we will not split our data into training and test sets. In 'real-world' projects, you would want to split your data to see how your model performs with data that it has never seen before.

In [3]:
# Remove the Example column from the dataframe
pie_df = pie_df.drop('Example', axis = 1)
pie_df

,Crust Shape,Crust Size,Crust Shade,Filling Size,Filling Shade,Class
0,Circle,Thick,Gray,Thick,Dark,pos
1,Circle,Thick,White,Thick,Dark,pos
2,Triangle,Thick,Dark,Thick,Gray,pos
3,Circle,Thin,White,Thin,Dark,pos
4,Square,Thick,Dark,Thin,White,pos
5,Circle,Thick,White,Thin,Dark,pos
6,Circle,Thick,Gray,Thick,White,neg
7,Square,Thick,White,Thick,Gray,neg
8,Triangle,Thin,Gray,Thin,Dark,neg
9,Circle,Thick,Dark,Thick,White,neg


In [4]:
# Using one-hot encoding to convert categorical variables into numerical ones. The drop_first=True argument specifies that the first binary column should be dropped, resulting in one less column than the number of unique values in the original column
features = pd.get_dummies(pie_df, columns = ['Crust Shape', 'Crust Size', 'Crust Shade', 'Filling Size', 'Filling Shade', 'Class'], drop_first=True)
features

,Crust Shape_Square,Crust Shape_Triangle,Crust Size_Thin,Crust Shade_Gray,Crust Shade_White,Filling Size_Thin,Filling Shade_Gray,Filling Shade_White,Class_pos
0,0,0,0,1,0,0,0,0,1
1,0,0,0,0,1,0,0,0,1
2,0,1,0,0,0,0,1,0,1
3,0,0,1,0,1,1,0,0,1
4,1,0,0,0,0,1,0,1,1
5,0,0,0,0,1,1,0,0,1
6,0,0,0,1,0,0,0,1,0
7,1,0,0,0,1,0,1,0,0
8,0,1,1,1,0,1,0,0,0
9,0,0,0,0,0,0,0,1,0


In [5]:
# Extracting the column 'Class_pos' from the dataframe from features and assigning to a new dataframe called response.
# Class_pos is a depenendent variable (or response variable) which will be used to predict using the Linear regression model.
response = features[['Class_pos']]
response

,Class_pos
0,1
1,1
2,1
3,1
4,1
5,1
6,0
7,0
8,0
9,0


## Perfrom Linear Regression Model Fitting

1. Import the [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) class from the `sklearn.linear_model` library. 

2. Instantiate an object of the `LinearRegression` class called `reg_model`.

3. Train the model by invoking the `fit()` method of the `reg_model` object and passing it `features` and `response`.

In [6]:
# Create an instance of LinearRegression and store it in a variable
from sklearn.linear_model import LinearRegression
reg_model = LinearRegression()
# Fit the model with the features and response
reg_model.fit(features, response)
reg_model.fit

<bound method LinearRegression.fit of LinearRegression()>

## Examine Linear Regression Model Parameters

View the trained model parameters by using the `coef_` and `intercept_` attributes of the trained model.

In [7]:
# Print the coefficients of the Linear Regression model
reg_model.coef_

array([[-2.30782836e-16, -2.17770284e-16, -2.03193627e-16,
        -7.39066573e-16, -4.25371534e-16,  7.11171324e-16,
        -6.49836083e-16, -9.28233668e-16,  1.00000000e+00]])

In [8]:
# Print the intercept of the Linear Regression model
reg_model.intercept_

array([8.8817842e-16])

## Making Predictions Using the Linear Regression Model

Evaluate the model's performance on the training data set by invoking the `predict()` method and passing `features` to it.  Save this output as `preds`. 


In [9]:
# Use the Linear Regression model to predict the response for the given features
preds = reg_model.predict(features)

Below are the results from the linear regression model:

The column "Class_pos" regards the "positive" or negative classification of the pie.  The column "Regression_Predictions" regards the predictions made by the linear regression model directly.  The column "Predicted_Responses" are the adjusted prdeictions made by the model after employing the cut-off values of 0 being 0 <= x <= 0.5 and 1 being 0.5 < x <= 1.0.

Note:  Make sure that your `response` is a DataFrame and not a Series or some of the code below may not function correctly.

In [10]:
# resp_comp = Response Comparison
resp_comp = response.copy() 
reg_outputs = [float(reg_model.predict(np.reshape(row, (1, -1)))) for row in features.itertuples(index=False)]
predicted_resp = np.array([1 if reg_output > 0.5 else 0 for reg_output in reg_outputs])
resp_comp = resp_comp.assign(Regression_Predictions = reg_outputs)
resp_comp = resp_comp.assign(Predicted_Responses = predicted_resp)
resp_comp

,Class_pos,Regression_Predictions,Predicted_Responses
0,1,1.000000e+00,1
1,1,1.000000e+00,1
2,1,1.000000e+00,1
3,1,1.000000e+00,1
4,1,1.000000e+00,1
5,1,1.000000e+00,1
6,0,-7.791218e-16,0
7,0,-4.178120e-16,0
8,0,4.393193e-16,0
9,0,-4.005525e-17,0


## Calculate Model Accuracy

Use the [accuracy_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) function to calculate the accuracy score of the model.  Save the accuracy score as `acc_score`.

In [11]:
# Import the accuracy_score function from sklearn library
from sklearn.metrics import accuracy_score
acc_score = accuracy_score(resp_comp['Class_pos'], resp_comp['Predicted_Responses'])
print("accuracy score: ", acc_score)

accuracy score:  1.0
